In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import datetime as dt
import warnings
warnings.filterwarnings('ignore')


In [36]:
df=pd.read_csv('train_MpHjUjU.csv',parse_dates=['MMM-YY','LastWorkingDate','Dateofjoining'])


In [37]:
df.head(20)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1
7,2017-02-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1
8,2017-03-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,350000,1
9,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,2,2,0,1


In [38]:
df.columns

Index(['MMM-YY', 'Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Total Business Value', 'Quarterly Rating'],
      dtype='object')

In [39]:
def ID_Group_Feat(df_ID_group):
    #calculating hike in salary 
    df_ID_group['sal_change']= df_ID_group['Salary']-df_ID_group['Salary'].shift(1)
    df_ID_group['sal_change']= df_ID_group['sal_change'].fillna(0)

    #did the employee got promotion
    df_ID_group['designation_change']= df_ID_group['Joining Designation']-df_ID_group['Designation']

    #change in ratings
    df_ID_group['rating_change']= df_ID_group['Quarterly Rating']-df_ID_group['Quarterly Rating'].shift(1)
    df_ID_group['rating_change']= df_ID_group['rating_change'].fillna(0)

    #months worked
    df_ID_group['months_worked']= (df_ID_group['MMM-YY']-df_ID_group['Dateofjoining']).dt.days/30
    #df_ID_group['months_worked']= round(df_ID_group['months_worked'],0)
    df_ID_group.loc[df_ID_group.months_worked < 0, 'months_worked'] = 0

    #caused loss
    df_ID_group['caused_loss']= np.where(df_ID_group['Total Business Value'].shift(1)<0, 1, 0)

    



    #Target 
    #df_ID_group['Target']=df_ID_group['Target'].sum()
    df_ID_group['Target']= np.where(df_ID_group['LastWorkingDate'].isnull(),0,1)
    df_ID_group['Target']= df_ID_group['Target'].sum()

    return df_ID_group
df=df.groupby('Emp_ID').apply(ID_Group_Feat)
df

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,sal_change,designation_change,rating_change,months_worked,caused_loss,Target
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2,0.0,0,0.0,0.266667,0,1
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2,0.0,0,0.0,1.300000,0,1
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,0.0,0,0.0,2.266667,1,1
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,0.0,0,0.0,0.000000,0,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaT,2,2,0,1,0.0,0,0.0,0.833333,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2017-08-01,2788,30,Male,C27,Master,70254,2017-06-08,NaT,2,2,740280,3,0.0,0,0.0,1.800000,0,0
19100,2017-09-01,2788,30,Male,C27,Master,70254,2017-06-08,NaT,2,2,448370,3,0.0,0,0.0,2.833333,0,0
19101,2017-10-01,2788,30,Male,C27,Master,70254,2017-06-08,NaT,2,2,0,2,0.0,0,-1.0,3.833333,0,0
19102,2017-11-01,2788,30,Male,C27,Master,70254,2017-06-08,NaT,2,2,200420,2,0.0,0,0.0,4.866667,0,0


In [40]:
cat_col= []
for col in df.columns:
    if df[col].dtype == 'O':
        cat_col.append(col)
cat_col


['Gender', 'City', 'Education_Level']

In [41]:
cols_to_drop= ['MMM-YY', 'Emp_ID','Dateofjoining', 'LastWorkingDate','Target']


In [42]:
num_cols=[col for col in df.columns if col not in cols_to_drop if col not in cat_col]

In [43]:
test= df[(df['MMM-YY']==pd.to_datetime('2017-12-01',format='%Y-%m-%d'))]
train= df[df['MMM-YY']<pd.to_datetime('2017-08-01',format='%Y-%m-%d')]

In [44]:
train

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,sal_change,designation_change,rating_change,months_worked,caused_loss,Target
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2,0.0,0,0.0,0.266667,0,1
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2,0.0,0,0.0,1.300000,0,1
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,0.0,0,0.0,2.266667,1,1
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1,0.0,0,0.0,0.000000,0,1
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1,0.0,0,0.0,0.833333,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19094,2016-04-01,2787,28,Female,C20,Master,69498,2015-07-21,NaT,1,1,0,1,0.0,0,-1.0,8.500000,0,1
19095,2016-05-01,2787,28,Female,C20,Master,69498,2015-07-21,NaT,1,1,0,1,0.0,0,0.0,9.500000,0,1
19096,2016-06-01,2787,28,Female,C20,Master,69498,2015-07-21,2016-06-20,1,1,0,1,0.0,0,0.0,10.533333,0,1
19097,2017-06-01,2788,29,Male,C27,Master,70254,2017-06-08,NaT,2,2,0,1,0.0,0,0.0,0.000000,0,0


In [45]:
X= train[cat_col+num_cols]
X[cat_col]= X[cat_col].astype('category')
y= train['Target']


In [46]:
test= df[df['MMM-YY']==pd.to_datetime('2017-12-01',format='%Y-%m-%d')]
X_test= test[cat_col+num_cols]
X_test[cat_col]= X_test[cat_col].astype('category')
y_test= test['Target']

In [47]:
from catboost import CatBoostClassifier

In [48]:
model = CatBoostClassifier(iterations=2,
                           learning_rate=1,
                           depth=2)
model.fit(X, y, cat_col)
preds = model.predict(X)

0:	learn: 0.5593694	total: 12.6ms	remaining: 12.6ms
1:	learn: 0.5290044	total: 28.6ms	remaining: 0us


In [49]:
from sklearn import metrics
metrics.f1_score(preds, y)

0.7893209518282065

In [50]:
params_cb={'learning_rate':0.02,'n_estimators':1000,'random_state':123,'cat_features':cat_col}
fold = StratifiedKFold(n_splits=5,shuffle=True,random_state=12345)
cb_scores,pred_cb,feat_scores=[],[],[]
for idxT, idxV in fold.split(X, y):

    df_trainX,df_trainY=X.iloc[idxT],y.iloc[idxT]
    df_evalX,df_evalY= X.iloc[idxV], y.iloc[idxV]
    
    cb=CatBoostClassifier(**params_cb,early_stopping_rounds=50,eval_metric='F1')
    cb.fit(df_trainX,df_trainY,eval_set=(df_evalX,df_evalY),plot=False, verbose=0)
    cb_scores.append(cb.get_best_score().get('validation').get('F1'))
    pred_cb.append(cb.predict_proba(df_evalX)[:,1])
    feat_scores.append(cb.get_feature_importance())
    print ('{} at {}'.format(cb.get_best_score().get('validation').get('F1'),cb.get_best_iteration()))
    
weights=cb_scores/sum(np.array(cb_scores))
print ('The local CV is {}'.format(np.sum(weights*cb_scores)))

0.951365409622887 at 993
0.9453792389334714 at 996
0.9528228924980665 at 986
0.9485655208064099 at 999
0.9485981308411213 at 993
The local CV is 0.9493532055339812


In [51]:
metrics.roc_auc_score(df_evalY,cb.predict(df_evalX))

0.9212520496851693

In [52]:
metrics.f1_score(df_evalY,cb.predict(df_evalX))

0.9485981308411213

In [53]:
featureImp=pd.DataFrame({'feature':df_trainX.columns,'importance':np.mean(np.array(feat_scores),0)})
featureImp=featureImp.sort_values('importance',ascending=False)
featureImp['importance']=featureImp['importance']*100/featureImp['importance'].sum()
featureImp.reset_index(drop=True)

,feature,importance
0,Salary,26.648192
1,City,16.803181
2,Age,12.951017
3,months_worked,9.380286
4,Education_Level,8.254783
5,Quarterly Rating,7.172090
6,designation_change,4.801538
7,Joining Designation,4.133122
8,Designation,4.091812
9,Gender,3.534441
